# Property Scrape

In [9]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [268]:
private_df = pd.read_csv('../data/raw/wcmsp-185935.csv')
private_df.head()

,Address,Postal Code,ENERGY STAR Score,Primary Property Type,Floor Area (Building) (ft²),Floor Area (Parking) (ft²),Year Built,Total GHG Emissions (Metric Tons CO2e),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Water Use (kgal),Unnamed: 13
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN


In [269]:
public_df = pd.read_csv('../data/raw/wcmsp-185933.csv')
public_df.head()

,Organization Name,Property Name,Address,Postal Code,ENERGY STAR Score,Primary Property Type,Floor Area (Buildings) (ft²),Floor Area (Parking) (ft²),Year Built,Total GHG Emissions (Metric Tons CO2e),Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Water Use (kgal),Unnamed: 15
0,Hennepin County,1800 Chicago,1800 Chicago S.,55404,88,Office,102815,0,1996,847,74,79,137,141,2574,NaN
1,Hennepin County,Central Library,300 Nicolet Mall,55401,0,Other,406354,266266,2004,2971,48,50,115,118,7203,NaN
2,Hennepin County,Century Plaza,330 south 12th st,55404,98,Office,297650,194471,1934,2375,56,58,127,130,2876,NaN
3,Hennepin County,Family Justice Center,110 S. 4th St.,55415,97,Courthouse,227422,49804,1956,1516,53,56,109,111,1087,NaN
4,Hennepin County,Forensic Sciences Building,530 Chicago Avenue South,55415,3,Medical Office,62602,0,1974,1219,218,224,331,339,401,NaN


In [302]:
import re
import string

def selenium_search_url(driver, search_url, address):
    driver.get(search_url)

    address_num = address.split(" ")[0]
    address_street = address.split(" ")[1:]
    address_box = driver.find_element_by_id("INPUT1")
    address_box.send_keys(address_num)
    street_box = driver.find_element_by_id("INPUT3")
    street_box.send_keys(address_street)

    submit_button = driver.find_element_by_id("INPUT4")
    submit_button.click()
    
    property_url = driver.current_url
    print(property_url)

    content = (driver.page_source).encode('ascii', 'ignore')
    return content

def create_soup(content):
    soup = BeautifulSoup(content,"lxml")
    return soup

def search_multiple_properties(address, trs, data):
    for tr in trs:
        match = re.search(r'(pid\=)(\d*)(\"\>)', str(tr))
        if match:
            print(match.group(2))
            url = "http://www16.co.hennepin.mn.us/pins/pidresult.jsp?pid=" + match.group(2)
            driver.get(url)
            content = driver.page_source
            soup = create_soup(content)
            parse_property_page(address, soup, data) 
    return data

def parse_property_page(address, soup, data):
    new_record = {}
    
    if "Direct Debit program" in soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[0].text.strip():
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[7].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = property_id 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[10].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[15].find_all('td')[1].text.strip()

        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[39].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[45].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[26].find_all('td')[1].text.strip()
            #positions with NO sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[44].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[47].find_all('td')[1].text.strip()
    
    elif "State Copy to be used when filing for the" in soup.find(id='TABLE_DETAILS').find_all('tr')[4].find_all('td')[0].text.strip():
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[8].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = property_id 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[11].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[16].find_all('td')[1].text.strip()
        
        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[46].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[26].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[27].find_all('td')[1].text.strip()
            #positions with sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[42].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[45].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[48].find_all('td')[1].text.strip()
    else:  
        property_id = int(soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text.strip().replace('-',''))
        new_record['address'] = address 
        new_record['property_id'] = soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text.strip() 
        new_record['construction_year'] = soup.find(id='TABLE_DETAILS').find_all('tr')[9].find_all('td')[4].text.strip() 
        new_record['owner_name'] = soup.find(id='TABLE_DETAILS').find_all('tr')[14].find_all('td')[1].text.strip()
        
        if "NO SALE INFORMATION ON FILE" in soup.find(id='TABLE_DETAILS').find_all('tr')[23].find_all('td')[0].text.strip():
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[38].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[39].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[44].find_all('td')[1].text.strip()
        else:
            new_record['sale_date'] = soup.find(id='TABLE_DETAILS').find_all('tr')[23].find_all('td')[1].text.strip()
            new_record['sale_price'] = soup.find(id='TABLE_DETAILS').find_all('tr')[24].find_all('td')[1].text.strip()
            new_record['transaction_type'] = soup.find(id='TABLE_DETAILS').find_all('tr')[25].find_all('td')[1].text.strip()
            #positions with NO sales section
            new_record['est_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[40].find_all('td')[1].text.strip()
            new_record['tax_market_value'] = soup.find(id='TABLE_DETAILS').find_all('tr')[41].find_all('td')[1].text.strip() 
            new_record['total_net_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[43].find_all('td')[1].text.strip() 
            new_record['total_tax'] = soup.find(id='TABLE_DETAILS').find_all('tr')[46].find_all('td')[1].text.strip()          
    
    new_record['land_market_value'] = soup.find(id='Table_Detail').find_all('tr')[3].find_all('td')[1].text.strip() 
    new_record['building_market_value'] = soup.find(id='Table_Detail').find_all('tr')[4].find_all('td')[1].text.strip() 
    new_record['total_market_value'] = soup.find(id='Table_Detail').find_all('tr')[6].find_all('td')[1].text.strip()
    new_record['homestead_mrkt_val_excl'] = soup.find(id='Table_Detail').find_all('tr')[9].find_all('td')[1].text.strip()
    new_record['property_type'] = soup.find(id='Table_Detail').find_all('tr')[11].find_all('td')[1].text.strip()
    new_record['homestead_status'] = soup.find(id='Table_Detail').find_all('tr')[12].find_all('td')[1].text.strip()

    data[property_id] = new_record
    return data

In [307]:
def run_scraper(df):
    chromedriver = "./chromedriver"
    driver = webdriver.Chrome(chromedriver)
    search_url="http://www16.co.hennepin.mn.us/pins/addrsrch.jsp"

    data = {}
    for index, row in df.iterrows():
        try:
            content = selenium_search_url(driver, search_url, row['Address'])
            soup = create_soup(content)

            if soup.find(id='TABLE_DETAILS') and soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text:
                print(soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text)
                parse_property_page(row['Address'], soup, data)
            elif soup.find(class_="notfound", string="No records found"):
                print("No records found for " +  row['Address'])
            elif soup.find("span", string="Click on property ID for details"):
                print(row['Address'] + " has multiple properties")
                select = driver.find_element_by_name("ps")
                select.send_keys('100')
                submit_button = driver.find_element_by_id("INPUT4")
                submit_button.click()

                content = (driver.page_source).encode('ascii', 'ignore')
                soup = create_soup(content)
                trs = soup.find(id='TABLE1').find_all('tr')
                search_multiple_properties(row['Address'], trs, data)
            else:
                print("Something went wrong " + row['Address'])
        except Exception:
            print("Exception on " +  row['Address'])
            pass
    return data
    

In [308]:
private_property_info = run_scraper(private_df)


http://www16.co.hennepin.mn.us/pins/addrresult.jsp
23-029-24-43-0394
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
No records found for 1011 Nicollet Mall
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
17-028-24-21-0100
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
22-028-24-34-0118
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
700 10Th Ave S has multiple properties
2602924240086
2602924240086
2402924240052
2402924240176
2402924240185
2402924240186
2402924240187
2402924240188
2402924240189
2402924240190
2402924240191
2402924240192
2402924240193
2402924240194
2402924240177
2402924240195
2402924240196
2402924240197
2402924240198
2402924240199
2402924240178
2402924240179
2402924240180
2402924240181
2402924240182
2402924240183
2402924240184
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
805 2Nd Ave S has multiple properties
2702924110085
2702924110085
2411722130087
http://www16.co.hennepin.mn.us/pins/addrresult.jsp
22-029-24-41-0085
http://www16.co.hennepin.mn.us/pin

KeyboardInterrupt: 

In [309]:
import pickle

with open('0129_private_prop_dict.pkl', 'wb') as picklefile:
    pickle.dump(private_property_info, picklefile)

In [ ]:
public_property_info = run_scraper(public_df)

In [310]:
# address  = "700 10Th Ave S"
    
# chromedriver = "./chromedriver"
# driver = webdriver.Chrome(chromedriver)
# search_url="http://www16.co.hennepin.mn.us/pins/addrsrch.jsp"

# data = {}

# # for index, row in private5.iterrows():
# content = selenium_search_url(driver, search_url, address)
# soup = create_soup(content)

# if soup.find(id='TABLE_DETAILS') and soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text:
#     print(soup.find(id='TABLE_DETAILS').find_all('tr')[6].find_all('td')[1].text)
#     parse_property_page(address, soup, data)
# elif soup.find(class_="notfound", string="No records found"):
#     print("No records found")
# elif soup.find("span", string="Click on property ID for details"):
#     print("multiple properties")
    
#     select = driver.find_element_by_name("ps")
#     select.send_keys('100')
#     submit_button = driver.find_element_by_id("INPUT4")
#     submit_button.click()
    
#     content = (driver.page_source).encode('ascii', 'ignore')
#     soup = create_soup(content)
#     trs = soup.find(id='TABLE1').find_all('tr')
#     search_multiple_properties(address, trs, data)
# else:
#     print("Something went wrong")

In [311]:
# chromedriver = "./chromedriver"
# driver = webdriver.Chrome(chromedriver)
# search_url="http://www16.co.hennepin.mn.us/pins/addrsrch.jsp"
# # driver.get("http://www16.co.hennepin.mn.us/pins/pidresult.jsp?pid=2302924340079")
# # content = (driver.page_source).encode('ascii', 'ignore')
# # soup = create_soup(content)
# address = "333 S 12th St"
# data = {}

# # parse_property_page(address, soup, data)

# selenium_search_url(driver, search_url, address)